# Solving a complex task with a multi-agent hierarchy

This notebook is part of the [Hugging Face Agents Course](https://www.hf.co/learn/agents-course), a free Course from beginner to expert, where you learn to build Agents.

![Agents course share](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/communication/share.png)

The reception is approaching! With your help, Alfred is now nearly finished with the preparations.

But now there's a problem: the Batmobile has disappeared. Alfred needs to find a replacement, and find it quickly.

Fortunately, a few biopics have been done on Bruce Wayne's life, so maybe Alfred could get a car left behind on one of the movie set, and re-engineer it up to modern standards, which certainly would include a full self-driving option.

But this could be anywhere in the filming locations around the world - which could be numerous.

So Alfred wants your help. Could you build an agent able to solve this task?

> 👉 Find all Batman filming locations in the world, calculate the time to transfer via a cargo plane to there, and represent them on a map, with a color varying by a cargo plane transfer time. Also represent some supercar factories with the same cargo plane transfer time.

Let's build this!

In [4]:
!pip install smolagents[litellm] plotly geopandas shapely kaleido -q


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import os
from huggingface_hub import login
from dotenv import load_dotenv

load_dotenv()
token = os.environ.get("HF_TOKEN")

login(token=token)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [8]:
# We first make a tool to get the cargo plane transfer time.
import math
from typing import Optional, Tuple

from smolagents import tool


@tool
def calculate_cargo_travel_time(
    origin_coords: Tuple[float, float],
    destination_coords: Tuple[float, float],
    cruising_speed_kmh: Optional[float] = 750.0,  # Average speed for cargo planes
) -> float:
    """
    Calculate the travel time for a cargo plane between two points on Earth using great-circle distance.

    Args:
        origin_coords: Tuple of (latitude, longitude) for the starting point
        destination_coords: Tuple of (latitude, longitude) for the destination
        cruising_speed_kmh: Optional cruising speed in km/h (defaults to 750 km/h for typical cargo planes)

    Returns:
        float: The estimated travel time in hours

    Example:
        >>> # Chicago (41.8781° N, 87.6298° W) to Sydney (33.8688° S, 151.2093° E)
        >>> result = calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093))
    """

    def to_radians(degrees: float) -> float:
        return degrees * (math.pi / 180)

    # Extract coordinates
    lat1, lon1 = map(to_radians, origin_coords)
    lat2, lon2 = map(to_radians, destination_coords)

    # Earth's radius in kilometers
    EARTH_RADIUS_KM = 6371.0

    # Calculate great-circle distance using the haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.asin(math.sqrt(a))
    distance = EARTH_RADIUS_KM * c

    # Add 10% to account for non-direct routes and air traffic controls
    actual_distance = distance * 1.1

    # Calculate flight time
    # Add 1 hour for takeoff and landing procedures
    flight_time = (actual_distance / cruising_speed_kmh) + 1.0

    # Format the results
    return round(flight_time, 2)


print(calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093)))

22.82


For the model provider, we use Together AI, one of the new [inference providers on the Hub](https://huggingface.co/blog/inference-providers)!

Regarding the GoogleSearchTool: this requires either having setup env variable `SERPAPI_API_KEY` and passing `provider="serpapi"` or having `SERPER_API_KEY` and passing `provider=serper`.

If you don't have any Serp API provider setup, you can use `WebSearchTool` but beware that it has a rate limit.

In [15]:
import os
from PIL import Image
from smolagents import CodeAgent, LiteLLMModel, GoogleSearchTool, InferenceClientModel, VisitWebpageTool

gemini_api_key = os.getenv("GEMINI_API_KEY")
model_id = os.getenv("MODEL_ID", "gemini/gemini-2.5-flash")
#model = InferenceClientModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct", provider="together")
model = LiteLLMModel(model_id=model_id, api_key=gemini_api_key)

We can start with creating a baseline, simple agent to give us a simple report.

In [8]:
task = """Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.
Also give me some supercar factories with the same cargo plane transfer time."""

In [11]:
#load SERPAPI_API_KEY from .env
from dotenv import load_dotenv
load_dotenv()
serper = os.environ.get("SERPER_API_KEY")

In [16]:
agent = CodeAgent(
    model=model,
    tools=[GoogleSearchTool(provider="serper"), VisitWebpageTool(), calculate_cargo_travel_time],
    additional_authorized_imports=["pandas"],
    max_steps=20,
)

In [17]:
result = agent.run(task)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ LiteLLMModel - gemini/gemini-2.5-flash ────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  batman_locations_raw = web_search(query="Batman filming locations")                                              
  print(batman_locations_raw)                                                                                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results
0. [The Batman | Film Locations](https://movie-locations.com/movies/b/The-Batman-2022-2.php)

The Batman filming location: St George's Hall, Liverpool · The Batman filming location: Royal Liver Building, Pier 
Head, Liverpool · The Batman filming location: ...

1. [The Batman (2022) - Filming & production](https://www.imdb.com/title/tt1877830/locations/)

Filming locations: Necropolis Cemetery, Glasgow, Scotland, UK (Batman and Selina leaving the cemetery), St. 
George's Hall, Liverpool, England, UK (Exterior ...

2. [Where was The Batman filmed? ALL the Filming Locations 
...](https://www.atlasofwonders.com/2022/04/where-was-the-batman-filmed.html)

The Batman was primarily filmed in the United Kingdom. Most of the recognizable buildings of this new version of 
Gotham City are located in Liverpool.

3. [Batman (1989) - Filming & production](https://www.imdb.com/title/tt0096895/locations/)

Filming locations ; Knebworth House, Knebworth, Hertfordshire, England, UK. (Wayne Manor; exterior) · 22 ; Acton 
Lane Power Station, Acton Lane, Acton, London, ...

4. [Batman Begins Filming Locations - 
Giggster](https://giggster.com/guide/movie-location/where-was-batman-begins-filmed)

Batman Begins was filmed in various locations across the globe, with most scenes shot in the United Kingdom, 
specifically in London, Buckinghamshire, and Essex.

5. [Category:Film Locations | Batman Wiki - Fandom](https://batman.fandom.com/wiki/Category:Film_Locations)

Category:Batman Returns Locations · Category:Batman Unlimited Locations · Category:Batman vs. Two-Face Locations · 
Category:The Batman (film) Locations.

6. [Batman Movies and Filming Locations in Los 
Angeles](https://www.traveltodayla.com/post/batman-movies-in-la?srsltid=AfmBOopfIRXNEeNBuo98sfAv6w2ixb1lCcYa8fNop4u
eBoQ_BpWQqBKQ)
Date published: Apr 20, 2023

Batman Movies and Filming Locations in LA · Cliffs of Marineland · Kirkeby Center · Gateway East Building · Wayne 
Manor · Bronson Caves · Batman ...

7. [What are the film locations of Batman 
movies?](https://www.facebook.com/groups/3164534807204454/posts/3749390682052194/)
Date published: 1 year ago

Batman (1989) Filming location: Wayne Manor (Hatfield House, Hertfordshire, UK) even after 35 years it looks the 
same!

8. [Dream Come True: Finding Dark Knight Filming Locations In ...](https://www.youtube.com/watch?v=8-df7RARK6I)
Date published: 8 months ago

Chicago Batman Tour: The Dark Knight Filming Locations! The last time I visited Chicago was 2006 before the Dark 
Knight came out in 2008.

Out: None

[Step 1: Duration 5.18 seconds| Input tokens: 2,481 | Output tokens: 420]

...

[Step 5: Duration 24.49 seconds| Input tokens: 31,967 | Output tokens: 3,310]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 6 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import pandas as pd                                                                                              
                                                                                                                   
  gotham_coords = (40.7128, -74.0060) # Gotham, New York, USA                                                      
                                                                                                                   
  # Batman Filming Locations and their selected coordinates                                                        
  batman_locations_coords = {                                                                                      
      "Liverpool, UK": (53.4072, -2.9917),                                                                         
      "Glasgow, UK": (55.86111, -4.25000),                                                                         
      "London, UK": (51.50722, -0.12750),                                                                          
      "Knebworth, UK": (51.8667, -0.1839),                                                                         
      "Chicago, USA": (41.88194, -87.62778),                                                                       
      "Los Angeles, USA": (34.052235, -118.243683),                                                                
  }                                                                                                                
                                                                                                                   
  all_locations_data = []                                                                                          
                                                                                                                   
  # Calculate travel times for Batman locations                                                                    
  for location_name, coords in batman_locations_coords.items():                                                    
      travel_time = calculate_cargo_travel_time(origin_coords=gotham_coords, destination_coords=coords)            
      all_locations_data.append({"Location": location_name, "Type": "Batman Filming", "Travel Time (hours)":       
  travel_time})                                                                                                    
                                                                                                                   
  # Supercar Factory Locations and their selected coordinates from the previous observation                        
  supercar_factories_coords = {                                                                                    
      "Bugatti Molsheim, France": (48.5268, 7.4997),                                                               
      "Koenigsegg Ängelholm, Sweden": (56.2911, 12.8546), # Using (56°17'28"N 12°51'16"E) approx from wikimapia    
      "Morgan Malvern, England": (52.123445, -2.310263),                                                           
      "Pagani Modena, Italy": (44.64694, 10.92556), # Using Modena city coordinates                                
      "Lamborghini Sant'Agata Bolognese, Italy": (44.66250, 11.13333),                                             
      "Porsche Zuffenhausen, Germany": (48.8345, 9.1510), # Using Porsche Museum coordinates                       
      "Ferrari Maranello, Italy": (44.526302, 10.866683),                                                          
  }                                                                                                                
                                                                                                                   
  # Calculate travel times for Supercar Factories        

Final answer:                                    Location              Type  \
0                             Liverpool, UK    Batman Filming   
1                               Glasgow, UK    Batman Filming   
2                                London, UK    Batman Filming   
3                             Knebworth, UK    Batman Filming   
4                              Chicago, USA    Batman Filming   
5                          Los Angeles, USA    Batman Filming   
6                  Bugatti Molsheim, France  Supercar Factory   
7              Koenigsegg Ängelholm, Sweden  Supercar Factory   
8                   Morgan Malvern, England  Supercar Factory   
9                      Pagani Modena, Italy  Supercar Factory   
10  Lamborghini Sant'Agata Bolognese, Italy  Supercar Factory   
11            Porsche Zuffenhausen, Germany  Supercar Factory   
12                 Ferrari Maranello, Italy  Supercar Factory   

    Travel Time (hours)  
0                  8.81  
1                  8.60  
2                  9.17  
3                  9.15  
4                  2.68  
5                  6.77  
6                 10.09  
7                 10.06  
8                  8.93  
9                 10.72  
10                10.74  
11                10.23  
12                10.72  

[Step 6: Duration 6.95 seconds| Input tokens: 52,150 | Output tokens: 4,138]

In [ ]:
result

,Location,Travel Time to Gotham (hours)
0,"Necropolis Cemetery, Glasgow, Scotland, UK",8.60
1,"St. George's Hall, Liverpool, England, UK",8.81
2,"Two Temple Place, London, England, UK",9.17
3,"Wollaton Hall, Nottingham, England, UK",9.00
4,"Knebworth House, Knebworth, Hertfordshire, Eng...",9.15
5,"Acton Lane Power Station, Acton Lane, Acton, E...",9.16
6,"Queensboro Bridge, New York City, USA",1.01
7,"Wall Street, New York City, USA",1.00
8,"Mehrangarh Fort, Jodhpur, Rajasthan, India",18.34
9,"Turda Gorge, Turda, Romania",11.89


We could already improve this a bit by throwing in some dedicated planning steps, and adding more prompting.

In [ ]:
agent.planning_interval = 4

detailed_report = agent.run(f"""
You're an expert analyst. You make comprehensive reports after visiting many websites.
Don't hesitate to search for many queries at once in a for loop.
For each data point that you find, visit the source url to confirm numbers.

{task}
""")

print(detailed_report)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You're an expert analyst. You make comprehensive reports after visiting many websites.                          │
│ Don't hesitate to search for many queries at once in a for loop.                                                │
│ For each data point that you find, visit the source url to confirm numbers.                                     │
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here is the plan of action that I will follow to solve the task:
                ```
                1. Perform a web search to find a comprehensive list of Batman filming locations worldwide.
2. For each filming location, visit the source URL to confirm the coordinates.
3. Calculate the cargo plane transfer time from each filming location to Gotham.
4. Perform a web search to find a comprehensive list of supercar factories worldwide.
5. For each supercar factory, visit the source URL to confirm the coordinates.
6. Calculate the cargo plane transfer time from each supercar factory to Gotham.
7. Filter both lists to include only locations with the same cargo plane transfer time.
8. Organize the filtered locations and their details into a pandas dataframe.
9. Provide the final answer as the pandas dataframe.


                ```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  Thought: I will perform a web search to find a comprehensive list of Batman filming locations worldwide.         
  Code:                                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
                                             Location  \
0          Necropolis Cemetery, Glasgow, Scotland, UK   
1           St. George's Hall, Liverpool, England, UK   
2               Two Temple Place, London, England, UK   
3              Wollaton Hall, Nottingham, England, UK   
4   Knebworth House, Knebworth, Hertfordshire, Eng...   
5   Acton Lane Power Station, Acton Lane, Acton, E...   
6               Queensboro Bridge, New York City, USA   
7                     Wall Street, New York City, USA   
8          Mehrangarh Fort, Jodhpur, Rajasthan, India   
9                         Turda Gorge, Turda, Romania   
10                                       Chicago, USA   
11                                   Hong Kong, China   
12  Cardington Studios, Northamptonshire, England, UK   
13  Warner Bros. Leavesden Studios, Hertfordshire,...   
14                     Westwood, Los Angeles, CA, USA   
15                               Woking, UK (McLaren)   

    Travel Time to Gotham (hours)  
0                            8.60  
1                            8.81  
2                            9.17  
3                            9.00  
4                            9.15  
5                            9.16  
6                            1.01  
7                            1.00  
8                           18.34  
9                           11.89  
10                           2.68  
11                          19.99  
12                           9.10  
13                           9.13  
14                           6.79  
15                           9.13

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 7 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import pandas as pd                                                                                              
                                                                                                                   
  # Manually specify coordinates for Batman filming locations                                                      
  batman_coordinates = {                                                                                           
      "St George's Hall, Liverpool, England, UK": (53.4084, -2.9916),                                              
      "Cardington Airship Hangar 1, Northamptonshire, England, UK": (52.2357, -0.4343),                            
      "Bridge of Sighs, Glasgow Necropolis, Glasgow, Scotland, UK": (55.8642, -4.2894),                            
      "Cherry Lane entrance, Anfield Cemetery, Liverpool, England, UK": (53.4084, -2.9916),                        
      "Wishart Street, Glasgow, Scotland, UK": (55.8642, -4.2894),                                                 
      "Chicago, USA": (41.8781, -87.6298),                                                                         
      "London, England, UK": (51.5074, -0.1278),                                                                   
      "Bedfordshire, England, UK": (52.1307, -0.4140),                                                             
      "Hong Kong, China": (22.3964, 114.1095)                                                                      
  }                                                                                                                
                                                                                                                   
  # Manually specify coordinates for supercar factories                                                            
  supercar_factories = {                                                                                           
      "Lamborghini, Sant'Agata Bolognese, Italy": (44.5403, 11.3467),                                              
      "Ferrari, Maranello, Italy": (44.5646, 10.8941),                                                             
      "Porsche, Leipzig, Germany": (51.3397, 12.3731),                                                             
      "McLaren, Woking, UK": (51.3774, -0.5713),                                                                   
      "Aston Martin, Gaydon, UK": (52.3705, -1.3104),                                                              
      "Koenigsegg, Ängelholm, Sweden": (56.1589, 12.8906),                                                         
      "Pagani, San Cesario sul Panaro, Italy": (44.7561, 10.9868),                                                 
      "Bugatti, Molsheim, France": (48.6922, 7.7486),                                                              
      "Rimac, Zagreb, Croatia": (45.8150, 15.9819),                                                                
      "Hypercar, Abu Dhabi, UAE": (24.4667, 54.3667)                                                               
  }                                                                                                                
                                                                                                                   
  # Gotham's coordinates                                                                                           
  gotham_coords = (40.7128, -74.0060)                                                                              
                                                                                                                   
  # Function to calculate cargo travel time                                                                        
  def calculate_travel_time(origin_coords, destination_co

Execution logs:
                                            Location  Travel Time (hours)
0  Bridge of Sighs, Glasgow Necropolis, Glasgow, ...                  8.6
1              Wishart Street, Glasgow, Scotland, UK                  8.6

Out - Final answer:                                             Location  Travel Time (hours)
0  Bridge of Sighs, Glasgow Necropolis, Glasgow, ...                  8.6
1              Wishart Street, Glasgow, Scotland, UK                  8.6

[Step 6: Duration 31.47 seconds| Input tokens: 83,622 | Output tokens: 5,145]

                                            Location  Travel Time (hours)
0  Bridge of Sighs, Glasgow Necropolis, Glasgow, ...                  8.6
1              Wishart Street, Glasgow, Scotland, UK                  8.6


In [19]:
agent.planning_interval = 4

detailed_report = agent.run(f"""
You're an expert analyst. You make comprehensive reports after visiting many websites.
Don't hesitate to search for many queries at once in a for loop.
For each data point that you find, visit the source url to confirm numbers.

{task}
""")

print(detailed_report)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You're an expert analyst. You make comprehensive reports after visiting many websites.                          │
│ Don't hesitate to search for many queries at once in a for loop.                                                │
│ For each data point that you find, visit the source url to confirm numbers.                                     │
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ LiteLLMModel - gemini/gemini-2.5-flash ────────────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here are the facts I know and the plan of action that I will follow to solve the task:
```
## 1. Facts survey

### 1.1. Facts given in the task
*   Origin for cargo transfer calculations: Gotham.
*   Coordinates for Gotham: 40.7128° N, 74.0060° W.
*   Output format requirement: pandas DataFrame.

### 1.2. Facts to look up
*   **Names of Batman films**: To ensure comprehensive coverage of filming locations.
    *   *Where to find*: Web search (e.g., "list of Batman movies", "Batman filmography").
*   **Filming locations for each Batman film**: Specific cities, countries, or notable landmarks where filming 
occurred.
    *   *Where to find*: Web search (e.g., "[Movie Title] filming locations", "where was [Movie Title] filmed"). 
URLs from search results, such as film databases (IMDb), travel blogs, or cinema news sites, will be visited to 
confirm.
*   **Geographical coordinates (latitude, longitude) for each identified Batman filming location**:
    *   *Where to find*: After identifying a specific location (e.g., "St Pancras International Station"), web 
search (e.g., "St Pancras International Station coordinates"). Google Maps or geographic information sites will be 
used to pinpoint precise coordinates.
*   **Names of prominent supercar manufacturers**: To identify factories.
    *   *Where to find*: Web search (e.g., "list of supercar brands", "top supercar manufacturers").
*   **Primary factory locations (address, city, country) for each identified supercar manufacturer**:
    *   *Where to find*: Web search (e.g., "[Manufacturer Name] factory location", "where is [Manufacturer Name] 
produced"). Official company websites, automotive news sites, or business directories will be visited to confirm.
*   **Geographical coordinates (latitude, longitude) for each identified supercar factory**:
    *   *Where to find*: After identifying a specific factory address, web search (e.g., "[Factory Address] 
coordinates"). Google Maps or geographic information sites will be used to pinpoint precise coordinates.

### 1.3. Facts to derive
*   **Cargo plane transfer time from Gotham to each Batman filming location**: Calculated using the 
`calculate_cargo_travel_time` tool with Gotham's coordinates as the origin and the coordinates of each filming 
location as the destination.
*   **Cargo plane transfer time from Gotham to each supercar factory**: Calculated using the 
`calculate_cargo_travel_time` tool with Gotham's coordinates as the origin and the coordinates of each factory as 
the destination.
*   **Structured dataset**: A consolidated collection of all derived data points, including location/factory name, 
type (Batman location/Supercar factory), latitude, longitude, and calculated travel time, structured appropriately 
for conversion into a pandas DataFrame.

## 2. Plan
1.  Perform web searches to identify a comprehensive list of Batman movies.
2.  For each identified Batman movie, perform web searches to find its primary filming locations.
3.  For each identified Batman filming location, perform web searches to obtain its precise geographical 
coordinates (latitude and longitude).
4.  Calculate the cargo plane travel time from Gotham's coordinates to each Batman filming location's coordinates 
using the `calculate_cargo_travel_time` tool.
5.  Perform web searches to identify a list of well-known supercar manufacturers.
6.  For each identified supercar manufacturer, perform web searches to find the address and precise geographical 
coordinates (latitude and longitude) of its main production factory.
7.  Calculate the cargo plane travel time from Gotham's coordinates to each supercar factory's coordinates using 
the `calculate_cargo_travel_time` tool.
8.  Aggregate all collected data (Batman filming locations and supercar factories, their coordinates, and 
calculated travel times) into a structured format suitable for a

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  batman_movies_search = web_search(query="list of Batman movies")                                                 
  print(batman_movies_search)                                                                                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results
0. [Batman in film](https://en.wikipedia.org/wiki/Batman_in_film)

ABC spinoff. Batman (1966) ; Original film series. Batman (1989); Batman Returns (1992); Batman Forever (1995); 
Batman & Robin (1997) ; The Dark Knight trilogy.

1. [All Batman films](https://www.imdb.com/list/ls051841763/)

1. The Dark Knight · 2. The Dark Knight Rises · 3. Joker · 4. Batman Begins · 5. Batman: The Dark Knight Returns, 
Part 2 · 6. Batman: Under the Red Hood · 7. Batman: ...

2. [Batman movies: chronological viewing timeline of DC 
...](https://batman-escape.com/en/movies/batman-movies-chronological-order)

Batman Begins; The Batman; The Dark Knight; The Dark Knight Rises; Batman (1989); Batman Returns; Batman Forever; 
Batman & Robin; Batman v ...

3. [BATMAN FILM HISTORY - Animated/Live-Action](https://batman-on-film.com/batman-films/)

THE BATMAN (1943) · BATMAN AND ROBIN (1949) · BATMAN (1966) · BATMAN (1989) · BATMAN RETURNS (1992) · BATMAN 
FOREVER (1995) · BATMAN & ROBIN (1997) · BATMAN BEGINS ( ...

4. [Every Relevant Batman Movie:, a list of films by 
Brauay](https://letterboxd.com/brauay/list/every-relevant-batman-movie/)
Date published: Apr 1, 2025

A list of 48 films compiled on Letterboxd, including Batman (1966), Batman (1989), Batman Returns (1992), Batman: 
Mask of the Phantasm ...

5. [Batman (franchise)](https://en.wikipedia.org/wiki/Batman_(franchise))

Batman (1943) · Batman and Robin (1949) · Batman (1966) · Batman (1989) · Batman Returns (1992) · Batman: Mask of 
the Phantasm (1993) · Batman Forever (1995) · Batman ...

6. [37 Batman Movies (Batman Forever), Ranked by 
...](https://editorial.rottentomatoes.com/guide/all-batman-movies-ranked/)

Batman returns with this list of classics, like The Dark Knight, Mask of the Phantasm, Batman Forever and more!

7. [How To Watch 'Batman' Movies In 
Order](https://www.forbes.com/sites/entertainment/article/batman-movies-in-order/)
Date published: Jan 8, 2024

Batman and Robin (1949) · Joker (1981) · Batman (1989) · Batman Forever (1995) · Batman & Robin (1997) · Batman 
Begins (2005) · The Dark Knight (2008).

8. [How to Watch All the Batman Movies in 
Order](https://www.esquire.com/entertainment/movies/a38982310/batman-movies-in-order/)
Date published: Aug 9, 2024

How to Watch All the Batman Movies in Order · What to Read Next · ADAM WEST ERA · Batman: The Movie (1966) · 
MICHAEL KEATON ERA · Batman (1989).

Out: None

...

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import pandas as pd                                                                                              
                                                                                                                   
  # Gotham coordinates                                                                                             
  GOTHAM_COORDS = (40.7128, -74.0060)                                                                              
                                                                                                                   
  # Pre-populated list of Batman filming locations with coordinates                                                
  batman_locations_data = [                                                                                        
      {"Location Name": "London", "City": "London", "Country": "England", "Latitude": 51.5074, "Longitude":        
  -0.1278},                                                                                                        
      {"Location Name": "Chicago", "City": "Chicago", "Country": "USA", "Latitude": 41.8781, "Longitude":          
  -87.6298},                                                                                                       
      {"Location Name": "Los Angeles", "City": "Los Angeles", "Country": "USA", "Latitude": 34.0522, "Longitude":  
  -118.2437},                                                                                                      
      {"Location Name": "New York City", "City": "New York City", "Country": "USA", "Latitude": 40.7128,           
  "Longitude": -74.0060}, # Gotham itself                                                                          
      {"Location Name": "Mehrangarh Fort", "City": "Jodhpur", "Country": "India", "Latitude": 26.2999,             
  "Longitude": 73.0185},                                                                                           
      {"Location Name": "Reykjavik Area", "City": "Reykjavik", "Country": "Iceland", "Latitude": 64.1265,          
  "Longitude": -21.8174},                                                                                          
      {"Location Name": "Glasgow", "City": "Glasgow", "Country": "Scotland", "Latitude": 55.8642[38;2;248;248;24

Execution logs:
Collected 10 Batman filming locations with travel times.
Collected 6 supercar factories with travel times.

Final answer:                        Type                     Location Name  \
0   Batman Filming Location                            London   
1   Batman Filming Location                           Chicago   
2   Batman Filming Location                       Los Angeles   
3   Batman Filming Location                     New York City   
4   Batman Filming Location                   Mehrangarh Fort   
5   Batman Filming Location                    Reykjavik Area   
6   Batman Filming Location                           Glasgow   
7   Batman Filming Location                           Detroit   
8   Batman Filming Location      Woking (McLaren Tech Centre)   
9   Batman Filming Location                         Hong Kong   
10         Supercar Factory                   Bugatti Factory   
11         Supercar Factory                   Ferrari Factory   
12         Supercar Factory  Porsche Headquarters and Factory   
13         Supercar Factory               Lamborghini Factory   
14         Supercar Factory         McLaren Technology Centre   
15         Supercar Factory         Pagani Automobili Factory   

                      City   Country  Latitude  Longitude  Travel Time (hours)  
0                   London   England   51.5074    -0.1278                 9.17  
1                  Chicago       USA   41.8781   -87.6298                 2.68  
2              Los Angeles       USA   34.0522  -118.2437                 6.77  
3            New York City       USA   40.7128   -74.0060                 1.00  
4                  Jodhpur     India   26.2999    73.0185                18.34  
5                Reykjavik   Iceland   64.1265   -21.8174                 7.17  
6                  Glasgow  Scotland   55.8642    -4.2518                 8.60  
7                  Detroit       USA   42.3314   -83.0458                 2.13  
8                   Woking   England   51.3408    -0.5423                 9.14  
9                Hong Kong     China   22.2855   114.1576                20.01  
10                Molsheim    France   48.5268     7.4997                10.09  
11               Maranello     Italy   44.5264    10.8667                10.72  
12               Stuttgart   Germany   48.8170     9.1670                10.23  
13    Sant'Agata Bolognese     Italy   44.6625    11.1333                10.74  
14                  Woking   England   51.3408    -0.5423                 9.14  
15  San Cesario sul Panaro     Italy   44.5888    11.0236                10.73  

[Step 10: Duration 17.67 seconds| Input tokens: 445,681 | Output tokens: 32,524]

                       Type                     Location Name  \
0   Batman Filming Location                            London   
1   Batman Filming Location                           Chicago   
2   Batman Filming Location                       Los Angeles   
3   Batman Filming Location                     New York City   
4   Batman Filming Location                   Mehrangarh Fort   
5   Batman Filming Location                    Reykjavik Area   
6   Batman Filming Location                           Glasgow   
7   Batman Filming Location                           Detroit   
8   Batman Filming Location      Woking (McLaren Tech Centre)   
9   Batman Filming Location                         Hong Kong   
10         Supercar Factory                   Bugatti Factory   
11         Supercar Factory                   Ferrari Factory   
12         Supercar Factory  Porsche Headquarters and Factory   
13         Supercar Factory               Lamborghini Factory   
14         Supercar Facto

Thanks to these quick changes, we obtained a much more concise report by simply providing our agent a detailed prompt, and giving it planning capabilities!

💸 But as you can see, the context window is quickly filling up. So **if we ask our agent to combine the results of detailed search with another, it will be slower and quickly ramp up tokens and costs**.

➡️ We need to improve the structure of our system.

## ✌️ Splitting the task between two agents

Multi-agent structures allow to separate memories between different sub-tasks, with two great benefits:
- Each agent is more focused on its core task, thus more performant
- Separating memories reduces the count of input tokens at each step, thus reducing latency and cost.

Let's create a team with a dedicated web search agent, managed by another agent.

The manager agent should have plotting capabilities to redact its final report: so let us give it access to additional imports, including `plotly`, and `geopandas` + `shapely` for spatial plotting.

In [6]:
import os
from smolagents import CodeAgent, LiteLLMModel, GoogleSearchTool, InferenceClientModel, VisitWebpageTool

In [9]:

from smolagents import LiteLLMModel
gemini_api_key = os.getenv("GEMINI_API_KEY")
model_i = os.getenv("MODEL_ID", "gemini/gemini-2.5-flash")
model = LiteLLMModel(model_id=model_i, api_key=gemini_api_key, max_tokens=8096)

web_agent = CodeAgent(
    model=model,
    tools=[
        GoogleSearchTool(provider="serper"),
        VisitWebpageTool(),
        calculate_cargo_travel_time,
    ],
    name="web_agent",
    description="Browses the web to find information",
    verbosity_level=0,
    max_steps=10,
)

The manager agent will need to do some mental heavy lifting.

So we give it the stronger model [DeepSeek-R1](https://huggingface.co/deepseek-ai/DeepSeek-R1), and add a `planning_interval` to the mix.

In [10]:
from dotenv import load_dotenv
load_dotenv()
openAI = os.environ.get("OPENAI_API_KEY")

In [ ]:
from smolagents.utils import encode_image_base64, make_image_url
from smolagents import OpenAIServerModel, LiteLLMModel


def check_reasoning_and_plot(final_answer, agent_memory):
    multimodal_model = OpenAIServerModel("gpt-4o", max_tokens=8096)
    filepath = ".\\img\\saved_map.png"
    assert os.path.exists(filepath), "Make sure to save the plot under saved_map.png!"
    image = Image.open(filepath)
    prompt = (
        f"Here is a user-given task and the agent steps: {agent_memory.get_succinct_steps()}. Now here is the plot that was made."
        "Please check that the reasoning process and plot are correct: do they correctly answer the given task?"
        "First list reasons why yes/no, then write your final decision: PASS in caps lock if it is satisfactory, FAIL if it is not."
        "Don't be harsh: if the plot mostly solves the task, it should pass."
        "To pass, a plot should be made using px.scatter_map and not any other method (scatter_map looks nicer)."
    )
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": make_image_url(encode_image_base64(image))},
                },
            ],
        }
    ]
    output = multimodal_model(messages).content
    print("Feedback: ", output)
    if "FAIL" in output:
        raise Exception(output)
    return True

gemini_api_key = os.getenv("GEMINI_API_KEY")
model_i_3 = os.getenv("MODEL_ID_3", "gemini/gemini-3-flash-preview")


manager_agent = CodeAgent(
    model= LiteLLMModel(model_id=model_i_3, api_key=gemini_api_key, max_tokens=8096),
    tools=[calculate_cargo_travel_time],
    managed_agents=[web_agent],
    additional_authorized_imports=[
        "geopandas",
        "plotly",
        "shapely",
        "json",
        "pandas",
        "numpy",
    ],
    planning_interval=5,
    verbosity_level=2,
    final_answer_checks=[check_reasoning_and_plot],
    max_steps=15,
)

Let us inspect what this team looks like:

In [28]:
manager_agent.visualize()

CodeAgent | gemini/gemini-2.5-flash
├── ✅ Authorized imports: ['geopandas', 'plotly', 'shapely', 'json', 'pandas', 'numpy']
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name                        ┃ Description                           ┃ Arguments                             ┃
│   ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ calculate_cargo_travel_time │ Calculate the travel time for a cargo │ origin_coords (`array`): Tuple of     │
│   │                             │ plane between two points on Earth     │ (latitude, longitude) for the         │
│   │                             │ using great-circle distance.          │ starting point                        │
│   │                             │                                       │ destination_coords (`array`): Tuple   │
│   │                             │                                       │ of (latitude, longitude) for the      │
│   │                             │                                       │ destination                           │
│   │                             │                                       │ cruising_speed_kmh (`number`):        │
│   │                             │                                       │ Optional cruising speed in km/h       │
│   │                             │                                       │ (defaults to 750 km/h for typical     │
│   │                             │                                       │ cargo planes)                         │
│   │ final_answer                │ Provides a final answer to the given  │ answer (`any`): The final answer to   │
│   │                             │ problem.                              │ the problem                           │
│   └─────────────────────────────┴───────────────────────────────────────┴───────────────────────────────────────┘
└── 🤖 Managed agents:
    └── web_agent | CodeAgent | Qwen/Qwen2.5-Coder-32B-Instruct
        ├── ✅ Authorized imports: []
        ├── 📝 Description: Browses the web to find information
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name                        ┃ Description                       ┃ Arguments                         ┃
            ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ web_search                  │ Performs a google web search for  │ query (`string`): The search      │
            │                             │ your query then returns a string  │ query to perform.                 │
            │                             │ of the top search results.        │ filter_year (`integer`):          │
            │                             │                                   │ Optionally restrict results to a  │
            │                             │                                   │ certain year                      │
            │ visit_webpage               │ Visits a webpage at the given url │ url (`string`): The url of the    │
            │                             │ and reads its content as a        │ webpage to visit.                 │
            │                             │ markdown string. Use this to      │                                   │
            │                             │ browse webpages.                  │                                   │
            │ calculate_cargo_travel_time │ Calculate the travel time for a   │ origin_coords (`array`): Tuple of │
            │                             │ cargo plane between two points on │ (latitude, longitude) for the     │
            │                             │ Earth using great-circle          │ starting point                    │
            │                             │ distance.     

In [ ]:
manager_agent.run("""
Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W).
Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in total.
Represent this as spatial map of the world, with the locations represented as scatter points with a color that depends on the travel time, and save it to saved_map.png!

Here's an example of how to plot and return a map:
import plotly.express as px
df = px.data.carshare()
fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,
     color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)
fig.show()
fig.write_image("saved_image.png")
final_answer(fig)

Never try to process strings using code: when you have a string to read, just print it and you'll see it.
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W).                                                                             │
│ Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in     │
│ total.                                                                                                          │
│ Represent this as spatial map of the world, with the locations represented as scatter points with a color that  │
│ depends on the travel time, and save it to saved_map.png!                                                       │
│                                                                                                                 │
│ Here's an example of how to plot and return a map:                                                              │
│ import plotly.express as px                                                                                     │
│ df = px.data.carshare()                                                                                         │
│ fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,      │
│      color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)                                    │
│ fig.show()                                                                                                      │
│ fig.write_image("saved_image.png")                                                                              │
│ final_answer(fig)                                                                                               │
│                                                                                                                 │
│ Never try to process strings using code: when you have a string to read, just print it and you'll see it.       │
│                                                                                                                 │
╰─ LiteLLMModel - gemini/gemini-2.5-flash ────────────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here are the facts I know and the plan of action that I will follow to solve the task:
```
## 1. Facts survey

### 1.1. Facts given in the task
*   **Origin location name:** Gotham
*   **Origin latitude:** 40.7128° N
*   **Origin longitude:** 74.0060° W
*   **Output map filename:** `saved_map.png`
*   **Minimum total locations required for the map:** 6 (combining Batman filming locations and supercar factory 
locations).

### 1.2. Facts to look up
*   **Specific names and geographical coordinates (latitude and longitude) of prominent Batman filming locations 
worldwide.**
    *   **Reasoning:** The task explicitly asks for "all Batman filming locations in the world." These are not 
provided and must be found using an external source. Accurate coordinates are essential for calculating travel 
times.
    *   **Source:** Web search using the `web_agent` tool. A suitable query would be "Batman filming locations 
world" or "list of Batman movie filming locations and their coordinates."
*   **Specific names and geographical coordinates (latitude and longitude) of supercar factories worldwide.**
    *   **Reasoning:** The task requires "some supercar factories with the same cargo plane transfer time" as the 
Batman locations, to reach a minimum of 6 points. These are not provided and must be found. Coordinates are 
necessary for calculations.
    *   **Source:** Web search using the `web_agent` tool. A suitable query would be "supercar manufacturing 
locations world" or "list of supercar factories and their locations."

### 1.3. Facts to derive
*   **Cargo plane travel time from Gotham (40.7128° N, 74.0060° W) to each identified Batman filming location.**
    *   **Reasoning:** The task explicitly asks for this calculation for each location. This requires the 
`calculate_cargo_travel_time` tool, which takes origin and destination coordinates.
*   **Cargo plane travel time from Gotham (40.7128° N, 74.0060° W) to each identified supercar factory location.**
    *   **Reasoning:** Similar to Batman locations, travel times must be calculated to fulfill the requirement of 
finding factories with "the same cargo plane transfer time" and for plotting. This also requires the 
`calculate_cargo_travel_time` tool.
*   **A collection of at least 6 unique locations (Batman filming locations and supercar factories) including their
names, geographical coordinates (latitude, longitude), calculated cargo plane travel times from Gotham, and a type 
identifier (e.g., 'Batman' or 'Supercar Factory').**
    *   **Reasoning:** This structured dataset is necessary to generate the `plotly.express.scatter_map` as 
specified in the task. The map requires latitude, longitude, and a color-encoding value (travel time). The "type" 
will help distinguish between location categories if needed for analysis or plotting.
*   **A `plotly.graph_objects.Figure` object representing the spatial map of the world with the derived 
locations.**
    *   **Reasoning:** The task explicitly requires generating this map with scatter points colored by travel time 
and saving it to `saved_map.png`.
*   **The `saved_map.png` image file.**
    *   **Reasoning:** The task explicitly states the map should be saved to this filename.

## 2. Plan

1.  Initialize an empty list to store all location data.
2.  Call `web_agent` to find "prominent Batman filming locations worldwide including their latitude and longitude".
3.  Process the `web_agent`'s output to extract the name, latitude, and longitude for each Batman filming location.
4.  For each extracted Batman location:
    a.  Call `calculate_cargo_travel_time` with Gotham's coordinates (40.7128, -74.0060) as origin and the Batman 
location's coordinates as destination.
    b.  Store the location name, its coordinates, the calculated travel time, and "Batman Filming Location" as its 
type in the location data list.
5.  Check

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
<code>                                                                                                             
all_locations_data = []                                                                                            
gotham_coords = (40.7128, -74.0060)                                                                                
                                                                                                                   
batman_locations_task = """                                                                                        
Find prominent Batman filming locations worldwide. For each location, provide its common name, precise latitude,   
and precise longitude.                                                                                             
Provide this information in a JSON format where the top-level key is "batman_locations" and its value is a list of 
objects.                                                                                                           
Each object in the list should have the following keys: "name", "latitude", and "longitude".                       
Ensure the latitude and longitude are given as numerical values (floats).                                          
Prioritize locations that have been frequently used or are iconic. Provide at least 5-7 distinct locations if      
possible.                                                                                                          
"""                                                                                                                
batman_locations_json_str = web_agent(task=batman_locations_task, additional_args={})                              
print(batman_locations_json_str)                                                                                   
                                                                                                                   

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  all_locations_data = []                                                                                          
  gotham_coords = (40.7128, -74.0060)                                                                              
                                                                                                                   
  batman_locations_task = """                                                                                      
  Find prominent Batman filming locations worldwide. For each location, provide its common name, precise           
  latitude, and precise longitude.                                                                                 
  Provide this information in a JSON format where the top-level key is "batman_locations" and its value is a list  
  of objects.                                                                                                      
  Each object in the list should have the following keys: "name", "latitude", and "longitude".                     
  Ensure the latitude and longitude are given as numerical values (floats).                                        
  Prioritize locations that have been frequently used or are iconic. Provide at least 5-7 distinct locations if    
  possible.                                                                                                        
  """                                                                                                              
  batman_locations_json_str = web_agent(task=batman_locations_task, additional_args={})                            
  print(batman_locations_json_str)                                                                                 
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Error in generating model output:
Model Qwen/Qwen2.5-Coder-32B-Instruct is not supported by provider together.

Code execution failed at line 'batman_locations_json_str = web_agent(task=batman_locations_task, 
additional_args={})' due to: AgentGenerationError: Error in generating model output:
Model Qwen/Qwen2.5-Coder-32B-Instruct is not supported by provider together.

[Step 1: Duration 2.83 seconds| Input tokens: 4,069 | Output tokens: 254]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



I don't know how that went in your run, but in mine, the manager agent skilfully divided tasks given to the web agent in `1. Search for Batman filming locations`, then `2. Find supercar factories`, before aggregating the lists and plotting the map.

Let's see what the map looks like by inspecting it directly from the agent state:

In [ ]:
manager_agent.python_executor.state["fig"]

![output map](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/unit2/smolagents/output_map.png)